In [1]:
# ! ls *.bed | parallel "annotatePeaks.pl {} hg19 > homer_anno/{.}.annoResults"
# /home/galaxy/project/alleleSpecific_analysis/results/ASm6A_addZr/stat_region/parse_annoResult.py

#### split into different genomic regions
# data_dir = "/Charles/project/ASm6A/ASm6A/homer_anno/"
data_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/homer_anno_100%/"
os.chdir(data_dir)

homer_list = glob.glob("*.annoResults")
result_dir = "split_region/"
os.system("mkdir -p %s" % result_dir)
####
def write_to_file(df):
    region, sample = df.iloc[0, 1], df.iloc[0, -1]
    sub_dir = os.path.join(result_dir, sample)
    os.system("mkdir -p %s" % sub_dir)
    df = df.iloc[:, :-2]
    res = os.path.join(sub_dir, "%s_SNPid.txt"%region)
    df.to_csv(res, sep="\t", index=False, header=False)
    
def split_into_region(in_file):
    df = pd.read_table(in_file, sep="\t")
    df['term'] = df['Chr'] + ":" + df['End'].astype(str)
    df['sample'] = os.path.basename(in_file).split(".")[0]
    df = df[['term', 'Annotation', 'sample']]
    df.columns = ['snpID','Annotation', 'sample']
    df['Annotation'] = df['Annotation'].str.split("(").str[0].str.strip().str.replace("' UTR", "UTR")
    df.groupby(['Annotation']).apply(write_to_file)

for homer in homer_list:
    split_into_region(homer)
#     bed_list = glob.glob("*.bed")
#     for bed in bed_list:
#         df = pd.read_table(bed)
#         an_list = df['annotation']
#         new_list = [x.split("(")[0] for x in an_list]
#         df['type'] = new_list
#         df['start'] = df['end'] - 1
#         df[['seqnames', 'start', 'end', 'type']].to_csv("format/%s"%bed, sep="\t", header=False, index=False)